Author: Daniel Abadjiev, with borrowed code from https://github.com/smart-pix/filter/blob/main/model_pipeline/model.py#L156 and from Eric You  
Date: Sep 25, 2025  
Description: Test out how hls4ml works

In [1]:
from OptimizedDataGenerator4 import *
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
# from qkeras import QDense, QActivation, QDenseBatchnorm
# from qkeras.quantizers import quantized_bits, quantized_relu
import hls4ml

noGPU=False
if noGPU:
    tf.config.set_visible_devices([], 'GPU')

print(tf.config.experimental.list_physical_devices())
print(tf.test.is_built_with_cuda())
print(tf.test.is_built_with_gpu_support())
print(tf.test.is_gpu_available())

import os
import numpy as np
import tensorflow as tf
import csv
import pandas as pd
# import model as md
# import utils as ut

# from qkeras import QDenseBatchnorm
import qkeras
print("\n\nhls4ml version: ",hls4ml.__version__)


2025-10-27 14:11:55.675464: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-27 14:11:55.677317: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-27 14:11:55.716414: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-27 14:11:55.717037: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-27 14:11:56.336563: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

WARN: Unable to import optimizer(s) from expr_templates.py: No module named 'sympy'
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
True
True
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
False


hls4ml version:  0.8.1


/home/dabadjiev/miniconda3/envs/newHLSEnviro/lib/python3.8/site-packages/hls4ml/converters/__init__.py:27: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!", stacklevel=1)
2025-10-27 14:11:57.589554: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2025-10-27 14:11:57.598275: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping 

In [6]:
filepath = "/home/dabadjiev/smartpixels_ml_dsabadjiev/erics_git/quantized_model1_results_20250922_162405/quantized_8w0i_8a0i/trial_1/quantized_mlp_quantized_8w0i_8a0i_trial1.keras"#"./DanielModels/model1.keras"
# qmodel_file = "/local/d1/smartpixLab/fermiModels/ds8l6_padded_noscaling_qkeras_foldbatchnorm_d58w4a8model.h5"
filepath = "/home/dabadjiev/smartpixels_ml_dsabadjiev/Muon_Collider_Smart_Pixels/daniel/model1Test4hls.h5"
filepath = "/home/youeric/PixelML/smart_pixels_ml/eric_training_models/Model3/combined_results_20250730_021907/quantized_model_3bit_0int/best_model_3bit_0int.h5"
filepath = "/home/dabadjiev/smartpixels_ml_dsabadjiev/Muon_Collider_Smart_Pixels/daniel/best_model_3bit_0int.h5"
filepath = "/home/dabadjiev/smartpixels_ml_dsabadjiev/Muon_Collider_Smart_Pixels/eric/model2_results_20251021_213803/models/model2_quantized_8bit.h5"
filepathModel2 = "/home/dabadjiev/smartpixels_ml_dsabadjiev/Muon_Collider_Smart_Pixels/eric/model2_results_20251022_114752/models/model2_quantized_6bit.h5"
filepathModel2 = "/home/dabadjiev/smartpixels_ml_dsabadjiev/Muon_Collider_Smart_Pixels/eric/model2_results_20251027_141314/models/model2_quantized_4bit.h5"
filepath=filepathModel2
# filepath = "./quantized_mlp_quantized_2w0i_2a0i_trial1.h5"
# filepath = "/home/dabadjiev/smartpixels_ml_dsabadjiev/Muon_Collider_Smart_Pixels/daniel/quantized_mlp_quantized_2w0i_2a0i_trial1.h5"
# filepath = ""
# filepath = ""
co = {}       
qkeras.utils._add_supported_quantized_objects(co)
quantizedModel = tf.keras.models.load_model(filepath,custom_objects=co,compile=True)
output_dir = "./hlsTmpModel2"

In [7]:
config = hls4ml.utils.config_from_keras_model(quantizedModel, granularity='name')
# Convert to an hls model

hls_model = hls4ml.converters.convert_from_keras_model(quantizedModel, hls_config=config, part = 'xc7z020clg400-1', output_dir=output_dir,backend="Vitis")
##Part number input in above line as part='xcu
# part='xcu250-figd2104-2L-e',
hls_model.write()

Interpreting Model
Topology:
Layer name: x_profile, layer type: InputLayer, input shapes: [[None, 21]], output shape: [None, 21]
Layer name: z_global, layer type: InputLayer, input shapes: [[None, 1]], output shape: [None, 1]
Layer name: y_profile, layer type: InputLayer, input shapes: [[None, 13]], output shape: [None, 13]
Layer name: y_local, layer type: InputLayer, input shapes: [[None, 1]], output shape: [None, 1]
Layer name: xz_concat, layer type: Concatenate, input shapes: [[None, 21], [None, 1]], output shape: [None, 22]
Layer name: yl_concat, layer type: Concatenate, input shapes: [[None, 13], [None, 1]], output shape: [None, 14]
Layer name: xz_dense1, layer type: QDense, input shapes: [[None, 22]], output shape: [None, 32]
Layer name: yl_dense1, layer type: QDense, input shapes: [[None, 14]], output shape: [None, 32]
Layer name: xz_relu1, layer type: Activation, input shapes: [[None, 32]], output shape: [None, 32]
Layer name: yl_relu1, layer type: Activation, input shapes: [[N

In [8]:
hls_model.build(csim=False, synth=True, cosim=True, validation=False, export=True, vsynth=True, reset=True )


****** Vitis HLS - High-Level Synthesis from C, C++ and OpenCL v2024.1 (64-bit)
  **** SW Build 5069499 on May 21 2024
  **** IP Build 5075265 on Wed May 22 21:45:21 MDT 2024
  **** SharedData Build 5076995 on Wed May 22 18:29:18 MDT 2024
  **** Start of session at: Mon Oct 27 14:17:57 2025
    ** Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.
    ** Copyright 2022-2024 Advanced Micro Devices, Inc. All Rights Reserved.

source /code/Xilinx_2024.1/Vitis_HLS/2024.1/scripts/vitis_hls/hls.tcl -notrace
INFO: [HLS 200-10] For user 'dabadjiev' on host 'kdplab01' (Linux_x86_64 version 5.14.0-570.52.1.el9_6.x86_64) on Mon Oct 27 14:17:59 CDT 2025
INFO: [HLS 200-10] In directory '/home/dabadjiev/smartpixels_ml_dsabadjiev/smart-pixels-ml/hlsTmpModel2'
Sourcing Tcl script 'build_prj.tcl'
INFO: [HLS 200-1510] Running: open_project -reset myproject_prj 
INFO: [HLS 200-10] Creating and opening project '/home/dabadjiev/smartpixels_ml_dsabadjiev/smart-pixels-ml/hlsTmpModel2/myproject_prj'.
INFO

{'CosimResults': [['0.257813']],
 'CSynthesisReport': {'TargetClockPeriod': '5.00',
  'EstimatedClockPeriod': '4.302',
  'BestLatency': '41',
  'WorstLatency': '41',
  'IntervalMin': '1',
  'IntervalMax': '1',
  'FF': '136114',
  'LUT': '239392',
  'BRAM_18K': '0',
  'DSP': '0',
  'URAM': '0',
  'AvailableBRAM_18K': '280',
  'AvailableDSP': '220',
  'AvailableFF': '106400',
  'AvailableLUT': '53200',
  'AvailableURAM': '0'},
 'VivadoSynthReport': {'LUT': '121557',
  'FF': '127735',
  'BRAM_18K': '0',
  'DSP48E': '0'},
 'CosimReport': {'RTL': 'Verilog',
  'Status': 'Pass',
  'LatencyMin': 41,
  'LatencyMax': 41,
  'IntervalMin': 0,
  'IntervalMax': 0,
  'LatencyAvg': 41.0,
  'IntervalAvg': 0.0}}